In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
from unsloth import FastVisionModel # FastLanguageModel for LLMs
import torch
from PIL import Image # Import Pillow for image loading

model, tokenizer = FastVisionModel.from_pretrained(
    "seasalt29/imageModelBig",
    load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)
FastVisionModel.for_inference(model) # Enable for inference!

instruction = "You are an expert therapist. Your role is to recognize the emotion from the photo. Return the recognized emotion in one word."
messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)


==((====))==  Unsloth 2025.2.12: Fast Mllama vision patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)

In [ ]:
def emotion(image_path):
  try:
    image = Image.open(image_path)
  except FileNotFoundError:
      print(f"Error: Image file not found at '{image_path}'. Please check the path.")
      exit()
  except Exception as e:
      print(f"Error loading image: {e}")
      exit()
  inputs = tokenizer(
    image, # Use the loaded custom image here
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
  ).to("cuda")
  _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                    use_cache = True, temperature = 1.5, min_p = 0.1)
  return _

In [ ]:
from unsloth import FastLanguageModel

model1, tokenizer1 = FastLanguageModel.from_pretrained(
        model_name = "seasalt29/model3", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 512,
        dtype = None,
        load_in_4bit = False,
    )

model1 = FastLanguageModel.for_inference(model1)


==((====))==  Unsloth 2025.2.12: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

In [ ]:
import re

def get_response_text(text):
  match = re.search(r'### Response:\s*(.*)', text)
  if match:
        return match.group(1)

In [ ]:
def prompt_maker(user_text, detected_emotion):
  alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

  ### Instruction:
  {}

  ### Input:
  {}

  ### Response:
  {}"""

  return alpaca_prompt.format(
        f"Detected facial emotion: {detected_emotion}\n\n. You are a helpful mental health counselling assistant that also considers the detected facial emotion of the user, please answer the mental health questions based on the patient's description. The assistant gives helpful, comprehensive, and appropriate answers to the user's questions.", # instruction
        user_text, # input
        "", # output - leave this blank for generation!
    )


In [ ]:
def generate_counselling_response(user_text, user_image):
    """
    Combines the user's text input with the detected facial emotion
    and generates a tailored mental health counselling response.
    """
    detected_emotion = emotion(user_image)

    # Build the combined prompt for the therapist model
    decode1=tokenizer.decode(detected_emotion[0])
    cleaned_text = re.sub(r'<\|.*?\|>', '', decode1).strip()
    decode2=cleaned_text.split()[-1] if cleaned_text else None
    #print(decode2)

    input1 = tokenizer1(
    [
        prompt_maker(user_text, decode2)
    ], return_tensors = "pt").to("cuda")
    output1 = model1.generate(**input1, max_new_tokens = 512, use_cache = True)
    return output1

In [ ]:
op= generate_counselling_response("I passed all my exams right now. I feel so happy and content.", "ha.jpeg")


happy<|eot_id|>


In [ ]:
res1 = tokenizer1.decode(op[0])
res1

"<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n  ### Instruction:\n  Detected facial emotion: happy\n\n. You are a helpful mental health counselling assistant that also considers the detected facial emotion of the user, please answer the mental health questions based on the patient's description. The assistant gives helpful, comprehensive, and appropriate answers to the user's questions.\n\n  ### Input:\n  I passed all my exams right now. I feel so happy and content.\n\n  ### Response:\n   I'm glad to hear that you're feeling happy and content after passing your exams. It's a significant accomplishment, and it's natural to feel a sense of relief and satisfaction. However, it's essential to remember that this is just one milestone in your life, and there will be many more challenges and opportunities ahead. It's important to continue working towards your go